## **1. Download dataset**
**Note:** If you can't download using gdown due to limited number of downloads, please download it manually and upload it to your drive, then copy it from the drive to colab.
```python
from google.colab import drive

drive.mount('/content/drive')
!cp /path/to/dataset/on/your/drive .
```

In [ ]:
# https://drive.google.com/file/d/1lWuq0COKnU9mCfMvTEq54DBLgAh3yYDx/view?usp=drive_link
!gdown 1lWuq0COKnU9mCfMvTEq54DBLgAh3yYDx

Traceback (most recent call last):
  File "/usr/local/bin/gdown", line 8, in <module>
    sys.exit(main())
  File "/usr/local/lib/python3.10/dist-packages/gdown/__main__.py", line 172, in main
    download(
  File "/usr/local/lib/python3.10/dist-packages/gdown/download.py", line 202, in download
    res = sess.get(url, stream=True, verify=verify)
  File "/usr/local/lib/python3.10/dist-packages/requests/sessions.py", line 602, in get
    return self.request("GET", url, **kwargs)
  File "/usr/local/lib/python3.10/dist-packages/requests/sessions.py", line 589, in request
    resp = self.send(prep, **send_kwargs)
  File "/usr/local/lib/python3.10/dist-packages/requests/sessions.py", line 703, in send
    r = adapter.send(request, **kwargs)
  File "/usr/local/lib/python3.10/dist-packages/requests/adapters.py", line 667, in send
    resp = conn.urlopen(
  File "/usr/local/lib/python3.10/dist-packages/urllib3/connectionpool.py", line 789, in urlopen
    response = self._make_request(
  File "

## **2. Install and import libraries**

In [2]:
!pip install -q transformers==4.41.2
!pip install -q bitsandbytes==0.43.1
!pip install -q accelerate==0.31.0
!pip install -q langchain==0.2.5
!pip install -q langchainhub==0.1.20
!pip install -q langchain-chroma==0.1.1
!pip install -q langchain-community==0.2.5
!pip install -q langchain_huggingface==0.0.3
!pip install -q python-dotenv==1.0.1
!pip install -q pypdf==4.2.0
!pip install -q numpy==1.24.4

^C
^C


  error: subprocess-exited-with-error
  
  × Getting requirements to build wheel did not run successfully.
  │ exit code: 1
  ╰─> [33 lines of output]
      Traceback (most recent call last):
        File "D:\anaconda\envs\RAG\Lib\site-packages\pip\_vendor\pyproject_hooks\_in_process\_in_process.py", line 353, in <module>
          main()
        File "D:\anaconda\envs\RAG\Lib\site-packages\pip\_vendor\pyproject_hooks\_in_process\_in_process.py", line 335, in main
          json_out['return_val'] = hook(**hook_input['kwargs'])
                                   ^^^^^^^^^^^^^^^^^^^^^^^^^^^^
        File "D:\anaconda\envs\RAG\Lib\site-packages\pip\_vendor\pyproject_hooks\_in_process\_in_process.py", line 112, in get_requires_for_build_wheel
          backend = _build_backend()
                    ^^^^^^^^^^^^^^^^
        File "D:\anaconda\envs\RAG\Lib\site-packages\pip\_vendor\pyproject_hooks\_in_process\_in_process.py", line 77, in _build_backend
          obj = import_module(mod_path)


In [6]:
!pip install -q transformers==4.41.2


In [7]:
import torch

from transformers import BitsAndBytesConfig
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_huggingface.llms import HuggingFacePipeline

from langchain.memory import ConversationBufferMemory
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_community.document_loaders import PyPDFLoader, TextLoader
from langchain.chains import ConversationalRetrievalChain

from langchain_chroma import Chroma
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser
from langchain import hub

d:\anaconda\envs\RAG\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## **3. Read pdf file**

In [8]:
Loader = PyPDFLoader
FILE_PATH = "a survey on multimodal sacarsm.pdf"
loader = Loader(FILE_PATH)
documents = loader.load()

print("Number of documents: ", len(documents))
documents[0]

Number of documents:  9


Document(metadata={'source': 'a survey on multimodal sacarsm.pdf', 'page': 0}, page_content='A Survey of Multimodal Sarcasm Detection\nShafkat Farabi1, Tharindu Ranasinghe2, Diptesh Kanojia3, Yu Kong4, Marcos Zampieri1\n1George Mason University, USA,\n2Lancaster University, UK\n3University of Surrey, UK,\n4Michigan State University, USA\nmfarabi@gmu.edu\nAbstract\nSarcasm is a rhetorical device that is used to convey\nthe opposite of the literal meaning of an utterance.\nSarcasm is widely used on social media and other\nforms of computer-mediated communication moti-\nvating the use of computational models to identify\nit automatically. While the clear majority of ap-\nproaches to sarcasm detection have been carried out\non text only, sarcasm detection often requires ad-\nditional information present in tonality, facial ex-\npression, and contextual images. This has led to\nthe introduction of multimodal models, opening the\npossibility to detect sarcasm in multiple modalities\nsuch as 

## **4. Initialize text splitter and split document**

In [9]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000,
                                               chunk_overlap=100)

In [10]:
docs = text_splitter.split_documents(documents)

print("Number of mini-documents: ", len(docs))
docs[0]

Number of mini-documents:  54


Document(metadata={'source': 'a survey on multimodal sacarsm.pdf', 'page': 0}, page_content='A Survey of Multimodal Sarcasm Detection\nShafkat Farabi1, Tharindu Ranasinghe2, Diptesh Kanojia3, Yu Kong4, Marcos Zampieri1\n1George Mason University, USA,\n2Lancaster University, UK\n3University of Surrey, UK,\n4Michigan State University, USA\nmfarabi@gmu.edu\nAbstract\nSarcasm is a rhetorical device that is used to convey\nthe opposite of the literal meaning of an utterance.\nSarcasm is widely used on social media and other\nforms of computer-mediated communication moti-\nvating the use of computational models to identify\nit automatically. While the clear majority of ap-\nproaches to sarcasm detection have been carried out\non text only, sarcasm detection often requires ad-\nditional information present in tonality, facial ex-\npression, and contextual images. This has led to\nthe introduction of multimodal models, opening the\npossibility to detect sarcasm in multiple modalities\nsuch as 

## **5. Initialize text vectorization (embedding)**

In [11]:
embedding = HuggingFaceEmbeddings()

d:\anaconda\envs\RAG\Lib\site-packages\huggingface_hub\file_download.py:147: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\APTS\.cache\huggingface\hub\models--sentence-transformers--all-mpnet-base-v2. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
d:\anaconda\envs\RAG\Lib\site-packages\huggingface_hub\file_download.py:1142: FutureWarning: `resume_download

## **6. Create vector database and retriever**

In [12]:
vector_db = Chroma.from_documents(documents=docs,
                                  embedding=embedding)

retriever = vector_db.as_retriever()

QUERY = "kiến trúc mới nhất"
result = retriever.invoke(QUERY)

print("Number of relevant documents: ", len(result))
result[0]

Number of relevant documents:  4


Document(metadata={'page': 7, 'source': 'a survey on multimodal sacarsm.pdf'}, page_content='speech. a multilingual perspective. In SEPLN, 2018.\n[Guet al., 2018 ]Yue Gu, Kangning Yang, Shiyu Fu,\nShuhong Chen, Xinyu Li, and Ivan Marsic. Hybrid atten-\ntion based multimodal network for spoken language clas-\nsification. In ACL, 2018.\n[Hazarika et al., 2018 ]Devamanyu Hazarika, Soujanya Po-\nria, Sruthi Gorantla, Erik Cambria, Roger Zimmermann,\nand Rada Mihalcea. CASCADE: Contextual sarcasm de-\ntection in online discussion forums. In COLING, 2018.\n[Joshi et al., 2016 ]Aditya Joshi, Pushpak Bhattacharyya,\nMark Carman, Jaya Saraswati, and Rajita Shukla. How\ndo cultural differences impact the quality of sarcasm anno-\ntation?: A case study of Indian annotators and American\ntext. In SIGHUM, 2016.\n[Joshi et al., 2017 ]Aditya Joshi, Pushpak Bhattacharyya,\nand Mark J Carman. Automatic sarcasm detection: A sur-\nvey. ACM Computing Surveys (CSUR), 50(5):1–22, 2017.\n[Khodak et al., 2018

## **7. Load LLMs (Vicuna)**

In [17]:
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline
import torch

MODEL_NAME = "lmsys/vicuna-7b-v1.5"

# Load the model and tokenizer
model = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME,
    quantization_config=nf4_config,
    low_cpu_mem_usage=True
)
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

# Create the pipeline
model_pipeline = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    max_new_tokens=512,
    pad_token_id=tokenizer.eos_token_id,
    device=0 if torch.cuda.is_available() else -1  # Use GPU if available, otherwise CPU
)

# Example usage of the pipeline
output = model_pipeline("Once upon a time,")
print(output)

RuntimeError: No GPU found. A GPU is needed for quantization.

## **8. Prompting with RAG**

In [ ]:
prompt = hub.pull("rlm/rag-prompt")

def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)



In [ ]:
USER_QUESTION = "Cách Cài đặt và sử dụng pre-trained model"
output = rag_chain.invoke(USER_QUESTION)
output

/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:515: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:520: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.6` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(


'Human: You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don\'t know the answer, just say that you don\'t know. Use three sentences maximum and keep the answer concise.\nQuestion: Cách Cài đặt và sử dụng pre-trained model \nContext: AI VIETNAM (AIO2024) aivietnam.edu.vn\nIV. Cài đặt chương trình và đánh giá\nTrong phần này, nhóm sẽ trình bày cách cài đặt, sử dụng và huấn luyện YOLOv10 trên bộ dữ\nliệu mới. Đồng thời, nhóm cũng thực hiện một thực nghiệm nhỏ nhằm so sánh hiệu suất của\nYOLOv10 so với hai phiên bản gần nhất là YOLOv8 và YOLOv9. Môi trường lập trình nhóm\nsử dụng là Google Colab.\nIV.I. Cài đặt và sử dụng pre-trained model\nMột cách nhanh chóng để sử dụng được YOLOv10 đó là sử dụng pre-trained model (mô hình đã\nđược huấn luyện sẵn trên bộ dữ liệu COCO - một bộ dữ liệu rất lớn). Để sử dụng pre-trained\nmodel, các bạn làm như sau:\n1.Cài đặt các thư viện cần thiết: Tải về mã nguồn của YOLOv10 và 

In [ ]:
answer = output.split('Answer:')[1].strip()
answer

'To use YOLOv10, you can download the pre-trained model from the GitHub repository and install the required libraries. Then, you can load the pre-trained model and test it on a new image.'